In [25]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import roc_auc_score
#1. Загрузите данные из файла data-logistic.csv. 
#Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

data = pd.read_csv('data-logistic.csv', header = None)
y = data[0]
x = data.loc[:,1:]

#2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, 
#что мы используем полноценный градиентный спуск, а не его стохастический вариант!

def w_1(w1,w2,y,x,k,C):
    l = len(y)
    summ = 0
    for i in range(0,l):
        summ += y[i] * x[1][i] * (1.0 - 1.0/(1.0 + math.exp(-y[i] * (w1*x[1][i] + w2*x[2][i]))))
    res = w1 + (k*(1.0/l)*summ) - k*C*w1
    return res
    
def w_2(w1,w2,y,x,k,C):
    l = len(y)
    summ = 0
    for i in range(0,l):
        summ +=y[i]*x[2][i]*(1.0 - 1.0/(1.0+math.exp(-y[i]*(w1*x[1][i]+w2*x[2][i]))))
    res = w2 +k*(1.0/l)*summ - k*C*w2
    return res

#3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10)
#логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте 
#вектор (0, 0).

#4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов 
#на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций 
#десятью тысячами.

def gradient(y,x, C ,w1 = 0.0,w2 = 0.0, k = 0.1, evk = 1e-5):
    l = len(y)
    i = 0
    max_i = 10000
    w1_new = w1
    w2_new = w2
    while i <= max_i:
        i += 1
        w1_new = w_1(w1,w2,y,x,k,C)
        w2_new = w_2(w1,w2,y,x,k,C)
        evklid_dist = np.linalg.norm(w1 - w1_new)+np.linalg.norm(w2 - w2_new) 
        
        if evklid_dist <= evk:
            break
        else:
            w1 = w1_new
            w2 = w2_new
            
    return [w1_new,w2_new]

w1,w2 = gradient(y,x, 0.0)
reg_w1, reg_w2 = gradient(y, x, 10.0)

#5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? 
#Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. 
#Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные
#обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: 
#a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

def sigmoid(x,w1,w2):
    a = 1.0 / (1.0 + math.exp(-w1*x[1] - w2*x[2]))
    return a

y_score = x.apply(lambda x: sigmoid(x,w1,w2), axis = 1)
y_reg_score = x.apply(lambda x: sigmoid(x, reg_w1, reg_w2), axis = 1)

auc = round(roc_auc_score(y, y_score),3)
reg_auc = round(roc_auc_score(y,y_reg_score),3)

print(auc,' ',reg_auc)

0.927   0.936
